In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import googlemaps
import time

In [25]:
# https://googlemaps.github.io/google-maps-services-python/docs/index.html
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)

In [26]:
victoria_bc_canada_coords = "48.407326,-123.329773"

def get_places_nearby(keyword):
    data = []
    page_token = None
    results = {"next_page_token": True}
    page = 1
    while 'next_page_token' in results:
        print(f"Searching: {keyword} | page {page}")
        results = gmaps.places_nearby(location=victoria_bc_canada_coords, radius=6000, keyword=keyword, page_token=page_token)
        
        if 'next_page_token' in results:
            page_token = results['next_page_token']
        
        for result in results['results']:
            row = {
                'place_id': result['place_id'],
                'name': result['name'],
                
                'avg_rating': result['rating'] if 'rating' in result else np.nan,
                
                'lat': result['geometry']['location']['lat'],
                'lng': result['geometry']['location']['lng'],
                
                'user_ratings_total': result['user_ratings_total'] if 'user_ratings_total' in result else np.nan,
            }
            data.append(row)
        time.sleep(2)
        page += 1

    return data

In [27]:
def get_place_details(place_data_df):
    for index, row in place_data_df.iterrows():
        place_details = gmaps.place(place_id=row['place_id'], 
                                   fields=['reviews', 'place_id'])
        
        if "reviews" in place_details['result']:
            review_blob = ""
            for review in place_details['result']['reviews']:
                review_blob += review['text']
            place_data_df.loc[index, 'reviews'] = review_blob
    
    return place_data_df


In [28]:
def get_place_data(searches):
    data = []
    for search in searches:
        data.extend(get_places_nearby(search))

    df = pd.DataFrame(data)
    print("Scraped: ", df.shape)

    df = df.drop_duplicates(subset=['name'], keep='first')
    print("Unique:", df.shape)

    df = get_place_details(df)

    return df

# Locations

In [29]:
# https://developers.google.com/maps/documentation/places/web-service/supported_types

location_searches = ["things to do", "tourist attractions", "scenic view points", "historical sites", "beaches", "clothing stores", "restaurants"]

In [30]:
df_locations = get_place_data(location_searches)

Searching: things to do | page 1
Searching: things to do | page 2
Searching: things to do | page 3
Searching: tourist attractions | page 1
Searching: tourist attractions | page 2
Searching: scenic view points | page 1
Searching: historical sites | page 1
Searching: historical sites | page 2
Searching: beaches | page 1
Searching: beaches | page 2
Searching: clothing stores | page 1
Searching: clothing stores | page 2
Searching: clothing stores | page 3
Searching: restaurants | page 1
Searching: restaurants | page 2
Searching: restaurants | page 3
Scraped:  (281, 6)
Unique: (222, 6)


In [31]:
df_locations.head(1)

,place_id,name,avg_rating,lat,lng,user_ratings_total,reviews
0,ChIJ1e5UJJF0j1QR6VpPNf8TYiQ,Royal BC Museum,4.6,48.419811,-123.36745,8658,Great museum has very nice and extensive exhib...


In [32]:
df_locations.to_csv("locations.csv", index=False)

# Hotels

In [33]:
# https://developers.google.com/maps/documentation/places/web-service/supported_types

hotel_searches = ["hotels", "hostels", "motels", "bed and breakfasts"]

In [34]:
df_hotels = get_place_data(hotel_searches)

Searching: hotels | page 1
Searching: hotels | page 2
Searching: hotels | page 3
Searching: hostels | page 1
Searching: motels | page 1
Searching: motels | page 2
Searching: motels | page 3
Searching: bed and breakfasts | page 1
Searching: bed and breakfasts | page 2
Scraped:  (145, 6)
Unique: (85, 6)


In [35]:
df_hotels.head(1)

,place_id,name,avg_rating,lat,lng,user_ratings_total,reviews
0,ChIJq9Z_wrBzj1QRTxNDocvoQk8,"The Vic, Ascend Hotel Collection",4.4,48.442703,-123.386117,82,Everything was amazing! The room was spotless ...


In [36]:
df_hotels.to_csv("hotels.csv", index=False)